In [1]:
from openai import OpenAI, AssistantEventHandler

client = OpenAI()
from typing_extensions import override

client = OpenAI()

In [ ]:

# Create a thread and attach the file to the message
thread = client.beta.threads.create(messages=[
    {"role": "user", "content": "How many shares of AAPL were outstanding at the end of of October 2023?", }])

# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

In [ ]:

class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    @override
    def on_message_done(self, message) -> None:
        # print a citation to the file searched
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        print(message_content.value)
        print("\n".join(citations))


# Then, we use the stream SDK helper
# with the EventHandler class to create the Run
# and stream the response.
assistant_id = "asst_49hfbuTeurnEcT0jd9KuNKyV"
thread_id = "thread_tEszx5NNXbL5nHIdj5DIw8Dw"
with client.beta.threads.runs.stream(thread_id=thread_id,
                                     assistant_id=assistant_id,
                                     instructions="",
                                     event_handler=EventHandler(),
                                     include=["step_details.tool_calls[*].file_search.results[*].content"]) as stream:
    stream.until_done()

In [8]:

run_id = "run_Rr27OeqOJgxfd8q5Lt3BuUpB"
thread_id = "thread_tEszx5NNXbL5nHIdj5DIw8Dw"
run_step = client.beta.threads.runs.steps.retrieve(thread_id=thread_id,
                                                   run_id=run_id,
                                                   step_id="step_BRs4NXzbDKN1HStETpqRCcSX",
                                                   include=[
                                                       "step_details.tool_calls[*].file_search.results[*].content"])

print(run_step)

RunStep(id='step_BRs4NXzbDKN1HStETpqRCcSX', assistant_id='asst_49hfbuTeurnEcT0jd9KuNKyV', cancelled_at=None, completed_at=1735001939, created_at=1735001937, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_Rr27OeqOJgxfd8q5Lt3BuUpB', status='completed', step_details=ToolCallsStepDetails(tool_calls=[FileSearchToolCall(id='call_Um1k14H2iNNudzMvqSKqw12n', file_search=FileSearch(ranking_options=FileSearchRankingOptions(ranker='default_2024_08_21', score_threshold=0.0), results=[FileSearchResult(file_id='file-XAov3nrJbk2byvhzCD282p', file_name='peplink_max_user_manual_peplink_documentation.pdf', score=0.8121620935585584, content=[FileSearchResultContent(text='PPTP. To configure this feature, navigate to Advanced > Remote User Access and choose the required VPN type.\n\nRemote User Access Settings\n\nEnable When this box is checked, this Remote User Access profile will be enabled.\n\nIf it is left unchecked, it will be disabled.\n\n\n\n173

In [ ]:
RunStep(id='step_BRs4NXzbDKN1HStETpqRCcSX',
        assistant_id='asst_49hfbuTeurnEcT0jd9KuNKyV',
        cancelled_at=None,
        completed_at=1735001939,
        created_at=1735001937,
        expired_at=None,
        failed_at=None,
        last_error=None,
        metadata=None,
        object='thread.run.step',
        run_id='run_Rr27OeqOJgxfd8q5Lt3BuUpB',
        status='completed',
        step_details=ToolCallsStepDetails(tool_calls=[FileSearchToolCall(id='call_Um1k14H2iNNudzMvqSKqw12n',
                                                                         file_search=FileSearch(ranking_options=FileSearchRankingOptions(
                                                                             ranker='default_2024_08_21',
                                                                             score_threshold=0.0),
                                                                                                results=[
                                                                                                    FileSearchResult(
                                                                                                        file_id='file-XAov3nrJbk2byvhzCD282p',
                                                                                                        file_name='peplink_max_user_manual_peplink_documentation.pdf',
                                                                                                        score=0.8121620935585584,
                                                                                                        content=[
                                                                                                            FileSearchResultContent(
                                                                                                                text='PPTP. To configure this feature, navigate to Advanced > Remote User Access and choose the required VPN type.\n\nRemote User Access Settings\n\nEnable When this box is checked, this Remote User Access profile will be enabled.\n\nIf it is left unchecked, it will be disabled.\n\n\n\n173\n\nVPN Type This field allows you to select the VPN type for the remote user access connection. The available options are:\xa0\n\nL2TP with IPsec\n\nIf L2TP with IPsec is selected, it may need to enter the pre-shared key for the remote user access.\xa0\n\nPPTP\xa0\n\nIf PPTP selected, there is no additional configuration required. The Point-to-Point Tunneling Protocol (PPTP) is an\n\nobsolete method for implementing virtual private networks. PPTP has many well known security issues\n\nOpenVPN\n\nIf the OpenVPN is selected, the OpenVPN Client\xa0 profile can be downloaded from the Status > Device page after\n\nthe configuration has been saved.\n\nYou have a choice between 2 different OpenVPN Client profiles:\n\n“Route all traffic” profile\n\nUsing this profile, VPN clients will send all the traffic through the OpenVPN tunnel\n\n“Split tunnel” profile\n\nUsing this profile, VPN clients will ONLY send those traffic designated to the untagged LAN and VLAN segment\n\nthrough the OpenVPN tunnel.\n\nPre-shared Key If L2TP with IPsec is selected in the VPN Type, enter the pre shared key in the text field. Please note that remote\n\ndevices will need this preshared key to access the Balance.\n\nDisabled Weak\n\nCiphers\n\nYou may click the \xa0button to show in the Pre-shared key and enable this option.\n\nWhen checked, weak ciphers such as 3DES will be disabled.\n\nPlease note: Legacy and Android devices may not able to connect.\n\nConnection Security\n\nRefresh\n\nIf OpenVPN is selected in the VPN Type, this settings is for specifying the interval for refreshing the connection.\n\nListen On This setting is for specifying the WAN IP addresses that allow remote user access.\n\nPort\xa0 If OpenVPN is selected in the VPN Type, the Port setting specifies the port(s) that correspond to the service.\xa0\n\n\n\n174\n\nAuthentication Determine the method of authenticating remote users:\n\nLocal User Accounts\n\nThis setting allows you to define the Remote User Accounts. Click Add to input username and password to\n\ncreate an account. After adding the user accounts, you can click on a username to edit the account password.\xa0\n\nNote:\n\nThe username must contain lowercase letters, numerics, underscore(_), dash(-), at sign(@), and period(.) only.\n\nThe password must be between 8 and 12 characters long\n\nLDAP Server\n\nEnter the matching LDAP server details to allow for LDAP server authentication.\n\nRadius Server\n\nEnter the matching Radius server details to allow for Radius server authentication.\n\nActive Diretory\n\nEnter the matching Active Directory details to allow for Active Directory server authentication.\n\n\n\n175\n\nCh24. Miscellaneous Settings\n\nThe miscellaneous settings include configuration for High Availability, Certificate Manager, service forwarding, service passthrough, GPS\n\nforwarding, GPIO, Groupe Networks and SIM Toolkit (depending the feature is supported on the model of Peplink router that is being used).\n\nHigh Availability\n\nMany Pepwave routers support high availability (HA) configurations via an open standard virtual router redundancy protocol (VRRP, RFC 3768).\n\nIn an HA configuration, two Pepwave routers provide redundancy and failover in a master-slave arrangement. In the event that the master unit\n\nis down, the slave unit becomes active. High availability will be disabled automatically where there is a drop-in connection configured on a LAN\n\nbypass port.\n\nIn the diagram, the WAN ports of each Pepwave router connect to the router and to the modem. Both Pepwave routers connect to the same',
                                                                                                                type='text')]),
                                                                                                    FileSearchResult(
                                                                                                        file_id='file-XAov3nrJbk2byvhzCD282p',
                                                                                                        file_name='peplink_max_user_manual_peplink_documentation.pdf',
                                                                                                        score=0.7835736043792331,
                                                                                                        content=[
                                                                                                            FileSearchResultContent(
                                                                                                                text='Name\n\nEnter a name to represent this Wi-Fi WAN connection.\n\nEnable Click the checkbox to toggle the on and off state of this connection.\n\nConnection Priority This option allows you to configure the WAN connection whether for normal daily usage or as a backup\n\nconnection only.\n\nIf Always-on is chosen, the WAN connection will be kept on continuously, regardless of the priority of other WAN\n\nconnections.\n\nIf Backup is chosen, the WAN connection will depend on other WAN connections. It will not be used when one or\n\nmore higher priority dependent WAN connections are connected.\n\nIndependent from\n\nBackup WANs\n\nIf this is checked, the connection will be working independent from other Backup WAN connections. Those in\n\nBackup Priority will ignore the status of this WAN connection, and will be used when none of the other higher\n\npriority connections are available.\n\nRouting Mode This option allows you to select the routing method to be used in routing IP frames via the WAN connection. The\n\nmode can be either NAT (Network Address Translation) or IP Forwarding.\n\nIn the case if you need to choose IP Forwarding for your scenario. Click the button to enable IP Forwarding.\n\nStandby State This setting specifies the state of the WAN connection while in standby. The available options are Remain\n\nConnected and Disconnect.\n\nReply to ICMP PING If this setting is disabled, the WAN connection will not respond to ICMP ping requests. By default, this setting is\n\nenabled.\n\nWi-Fi WAN Settings\n\nChannel Width Select the channel width for this Wi-Fi WAN. 20MHz will have greater support for older devices using 2.4Ghz, while\n\n40MHz is appropriate for networks with newer devices that connect using 5Ghz\n\n\n\n117\n\nChannel Determine whether the channel will be automatically selected. If you select custom, the following table will appear:\n\nOutput Power If you are setting up a network with many Wi-Fi devices in close proximity, then you can configure the output power\n\nhere. Click the “boost” button for additional power. However, with that option ticked, output power may exceed local\n\nregulatory limits.\n\nData Rate Selecting Auto will enable the router to automatically determine the best data rate, while manually selecting a rate\n\nwill force devices to connect using the fixed rate.\xa0\n\nRoaming Checking this box will enable Wi-Fi roaming. Click the  icon for additional options.\n\nConnect to Any\n\nOpen Mode AP\n\nThis option is to specify whether the Wi-Fi WAN will connect to any open mode access points it finds.\n\nBeacon Miss\n\nCounter\n\nThis sets the threshold for the number of missed beacons.\n\nChannel Scan\n\nInterval\n\nConfigure Channel Scan Interval in ms.\n\nCreating Wi-Fi Connection Profiles\n\nYou can manually create a profile to connect to a Wi-Fi connection. This is useful for creating a profile for connecting to hidden-SSID access\n\npoints. Click Network > Wi-Fi WAN > Create Profile… to get started.\n\nThis will open a window similar to the one shown below\n\n\n\n118\n\nWi-Fi Connection Profile Settings\n\nNetwork Name (SSID) Enter a name to represent this Wi-Fi connection.\n\nSecurity This option allows you to select which security policy is used for this wireless network. Available options:\n\nOpen\n\nWEP\n\nEnhanced Open (OWE)\n\nWPA3 -Personal\n\nWPA2/WPA3 -Personal\n\nWPA/ WPA2 – Personal\n\nWPA/ WPA2 – ENterprise\n\n802.1X with dynamic WEP key\n\nShared Key Enter the password for the wireless network.\n\nPreffered BSSID Configure the BSSID. The BSSID is the MAC address of the wireless access point (WAP).\n\nConnected Method Choose DHCP or Static IP for the Wi-Fi WAN connection method.\n\nDNS Servers Configure the DNS servers that this WAN connection should use.\n\nWAN Connection Settings (Common)',
                                                                                                                type='text')]),
                                                                                                    FileSearchResult(
                                                                                                        file_id='file-XAov3nrJbk2byvhzCD282p',
                                                                                                        file_name='peplink_max_user_manual_peplink_documentation.pdf',
                                                                                                        score=0.7259963870527922,
                                                                                                        content=[
                                                                                                            FileSearchResultContent(
                                                                                                                text='Web Administration Settings (on External AP)\n\nEnable Check the box to allow the Pepwave router to manage the web admin access information of the AP.\n\nWeb Access Protocol These buttons specify the web access protocol used for accessing the web admin of the AP. The two available\n\noptions are HTTP and HTTPS.\n\nManagement Port This field specifies the management port used for accessing the device.\n\nHTTP to HTTPS\n\nRedirection\n\nThis option will be available if you have chosen HTTPS as the Web Access Protocol. With this enabled, any HTTP\n\naccess to the web admin will redirect to HTTPS automatically.\n\n\n\n204\n\nAdmin User Name This field specifies the administrator username of the web admin. It is set as admin by default.\n\nAdmin Password This field allows you to specify a new administrator password. You may also click the Generate button and let the\n\nsystem generate a random password automatically.\n\nThis allows users to configure AP Time Settings (both Timezone and NTP) in AP Controller.\n\nAP Time Settings\n\nTime Zone This field is to select the time zone for the AP controller.\xa0\n\nTime Server This field is to select the time server for the AP controller.\n\nThis settings is to allow user to manage external AP’s controller unreachable action. When Manage Unreachable Action is checked, there will\n\nhave 2 options which are “None” and “Radio Off“.\xa0\n\nThis is an option to enable client load balancing for AP Controller. When the option is enabled, it is trying to balance the station count on APs\n\nwithin the same profile.\xa0\n\nSome Pepwave models displays a screen similar to the one shown below, navigating to AP > Settings:\xa0\n\nWi-Fi Radio Settings\n\nOperating Country This option sets the country whose regulations the Pepwave router follows.\xa0\n\nWi-Fi Antenna Wi-Fi Antenna Choose from the router’s internal or optional external antennas, if so equipped.\xa0\n\nWi-Fi AP Settings\n\n\n\n205\n\nProtocol This option allows you to specify whether 802.11b and/or 802.11g client association requests will be accepted.\n\nAvailable options are 802.11ng and 802.11na. By default, 802.11ng is selected.\xa0\n\nChannel This option allows you to select which 802.11 RF channel will be used. Channel 1 (2.412 GHz) is selected by default.\n\nChannel Width Auto (20/40 MHz) and 20 MHz are available. The default setting is Auto (20/40 MHz), which allows both widths to be\n\nused simultaneously.\xa0\n\nOutput Power This option is for specifying the transmission output power for the Wi-Fi AP. There are 4 relative power levels available\n\n– Max, High, Mid, and Low. The actual output power will be bound by the regulatory limits of the selected country.\xa0\n\nBeacon RateA This option is for setting the transmit bit rate for sending a beacon. By default, 1Mbps is selected.\xa0\n\nBeacon\n\nIntervalA\n\nThis option is for setting the time interval between each beacon. By default, 100ms is selected.\xa0\n\nDITMA This field allows you to set the frequency for the beacon to include a delivery traffic indication message. The interval is\n\nmeasured in milliseconds. The default value is set to 1 ms.\xa0\n\nSlot TimeA This field is for specifying the wait time before the Router transmits a packet. By default, this field is set to 9 µs.\n\nACK TimeA This field is for setting the wait time to receive an acknowledgement packet before performing a retransmission. By\n\ndefault, this field is set to 48 µs.\xa0\n\nFrame\n\nAggreagtionA\n\nThis option allows you to enable frame aggregation to increase transmission throughput.\xa0\n\nGuard IntervalA This setting allows choosing a short or long guard period interval for your transmissions.\xa0\n\nCh26. AP Controller Status',
                                                                                                                type='text')]),
                                                                                                    FileSearchResult(
                                                                                                        file_id='file-XAov3nrJbk2byvhzCD282p',
                                                                                                        file_name='peplink_max_user_manual_peplink_documentation.pdf',
                                                                                                        score=0.6893626255793484,
                                                                                                        content=[
                                                                                                            FileSearchResultContent(
                                                                                                                text='Available options are 802.11ng and 802.11na.\xa0By default, 802.11ng is selected.\xa0\n\n\n\n202\n\nChannel Width There are three options: 20 MHz, 20/40 MHz, and 40 MHz. With this feature enabled, the Wi-Fi system can use two\n\nchannels at once. Using two channels improves the performance of the Wi-Fi connection.\n\nChannel This drop-down menu selects the 802.11 channel to be utilized. Available options are from 1 to 11 and from 1 to 13\n\nfor the North America region and Europe region, respectively. (Channel 14 is only available when the country is\n\nselected as Japan with protocol 802.11b.) If Auto is set, the system will perform channel scanning based on the\n\nscheduled time set and choose the most suitable channel automatically.\n\nAuto Channel\n\nUpdate\n\nIndicate the time of day at which update automatic channel selection.\n\nOutput Power This drop-down menu determines the power at which the AP under this profile will broadcast. When fixed settings\n\nare selected, the AP will broadcast at the specified power level, regardless of context. When Dynamic settings are\n\nselected, the AP will adjust its power level based on its surrounding APs in order to maximize performance.\xa0\n\nThe Dynamic: Auto setting will set the AP to do this automatically. Otherwise, the Dynamic: Manual setting will set\n\nthe AP to dynamically adjust only if instructed to do so. If you have set Dynamic:Manual, you can go to\n\nAP>Toolbox>Auto Power Adj. to give your AP further instructions.\n\nIf you click the Boost checkbox, the AP under this profile will transmit using additional power. Please note that\n\nusing this option with several APs in close proximity will lead to increased interference.\n\nClient Signal\n\nStrength Threshold\n\nThis field determines that maximum signal strength each individual client will receive. The measurement unit is\n\nmegawatts.\n\nMax number of\n\nClients\n\nThis field determines the maximum clients that can be connected to APs under this profile.\xa0\n\nManagement VLAN\n\nID\n\nThis field specifies the VLAN ID to tag to management traffic, such as AP to AP controller communication traffic. The\n\nvalue is 0 by default, meaning that no VLAN tagging will be applied.\n\nNote: change this value with caution as alterations may result in loss of connection to the AP controller.\n\nDiscover Nearby\n\nNetworksA\n\nThis option is to turn on and off to scan the nearby the AP.\xa0\n\nNote: Feature will be automatically turned on with Auto Channel / Dynamic Output Power\n\nBeacon RateA This drop-down menu provides the option to send beacons in different transmit bit rates. The bit rates are 1Mbps,\n\n2Mbps, 5.5Mbps, 6Mbps, and 11Mbps.\n\nBeacon IntervalA This drop-down menu provides the option to set the time between each beacon send. Available options are 100ms,\n\n250ms, and 500ms.\n\nDTIMA This field provides the option to set the frequency for beacon to include delivery traffic indication message (DTIM).\n\nThe interval unit is measured in milliseconds.\n\nRTS ThresholdA This field provides the option to set the minimum packet size for the unit to send an RTS using the RTS/CTS\n\nhandshake. Setting 0 disables this feature.\n\nFragmentation\n\nThresholdA\n\nDetermines the maximum size (in bytes) that each packet fragment will be broken down into. Set 0 to disable\n\nfragmentation.\n\nDistance/Time\n\nConverterA\n\nSelect the distance you want your Wi-Fi to cover in order to adjust the below parameters. Default values are\n\nrecommended.\n\nSlot TimeA This field provides the option to modify the unit wait time before it transmits. The default value is 9μs.',
                                                                                                                type='text')]),
                                                                                                    FileSearchResult(
                                                                                                        file_id='file-XAov3nrJbk2byvhzCD282p',
                                                                                                        file_name='peplink_max_user_manual_peplink_documentation.pdf',
                                                                                                        score=0.6839691466718577,
                                                                                                        content=[
                                                                                                            FileSearchResultContent(
                                                                                                                text='WAN DNS\n\nServers\n\nEnter the selected WAN’s corresponding DNS server IP addresses.\n\nA – Advanced feature, please click\xa0the  button on the top right-hand corner to activate.\n\n\n\n99\n\nPort Settings\n\nTo configure port settings, navigate to Network > Port Settings\n\nOn this screen, you can enable specific ports, as well as determine the speed of the LAN ports, whether each port is a trunk or access port, as\n\nwell as which VLAN each link belongs to, if any.\n\nCaptive Portal\n\nThe captive portal serves as a gateway that clients have to pass if they wish to access the internet using your router. To configure, navigate to\n\nNetwork > LAN > Captive Portal.\n\nCaptive Portal Settings\n\nName Enter the name for the Captive Portal.\n\n\n\n100\n\nEnable Check Enable and then, optionally, select the LANs/VLANs that will use the captive portal.\xa0\n\nHostname To customize the portal’s form submission and redirection URL, enter a new URL in this field. To reset the URL to\n\nfactory settings, click Default.\n\nAccess Mode Click Open Access to allow clients to freely access your router. Click User Authentication to force your clients to\n\nauthenticate before accessing your router.\xa0\n\nSelect External Server to use the Captive Portal with a HotSpot system.\n\nAs described in the following knowledgebase article:\n\nhttps://forum.peplink.com/t/using-hotspotsystem-wi-fi-on-pepwave-max-routers/\n\n(https://forum.peplink.com/t/using-hotspotsystem-wi-fi-on-pepwave-max-routers/)\n\nAuthentication When selecting the “User Authentication” in the Access Mode field, you will see the available option for the\n\nAuthentication via drop-down list:\xa0\n\nRADIUS Server\n\nLDAP Server\n\nFill in the necessary information to complete your connection to the server and enable authentication.\n\nhttps://forum.peplink.com/t/using-hotspotsystem-wi-fi-on-pepwave-max-routers/\nhttps://forum.peplink.com/t/using-hotspotsystem-wi-fi-on-pepwave-max-routers/\n\n\n101\n\nExternal Server When selecting the “External Server” in the Access Mode field, you will see the available option for the Service Type\n\nvia drop-down list:\n\nCoovaChilli\n\nHotspotSystem\n\nFill in the necessary information to complete your connection to the server and enable authentication.\n\nAccess Quota Set a time and data cap to each user’s Internet usage.\n\nQuota Reset Time This menu determines how your usage quota resets. Setting it to Daily will reset it at a specified time every day.\n\nSetting a number of minutes after quota reached establish a timer for each user that begins after the quota has\n\nbeen reached.\n\nInactive Timeout Clients will get disconnected when the inactive the configured time is reached.\n\nDefault 0: no timeout\n\nAllowed\n\nNetworks\n\nAdd networks that can bypass the captive Portal in this field.\n\nTo whitelist a network, enter the domain name / IP address here and click . To delete an existing network from\n\nthe list of allowed networks, click the button next to the listing.\n\nAllowed Clients Add MAC address and /or IP addresses for client devices that are allowed to bypass the Captive Portal. Clients\n\naccessing these domains and IP addresses will not be redirected to the splash page.\n\nSplash Page Here, you can choose between using the Pepwave router’s built-in captive portal and redirecting clients to a URL you\n\ndefine.\n\nPopup Handling Configurable options for popup handling:\n\n– Bypass Popup (Redirection only takes place on normal browser)\xa0–\xa0 Automatically show splash page on Safari for\n\nApple (iOS / macOS) devices\n\nLogout Hostname A hostname that can be used to logout captive portal when being accessed on browser.\n\nCustomize splash\n\npage',
                                                                                                                type='text')]),
                                                                                                    FileSearchResult(
                                                                                                        file_id='file-XAov3nrJbk2byvhzCD282p',
                                                                                                        file_name='peplink_max_user_manual_peplink_documentation.pdf',
                                                                                                        score=0.6833159604528638,
                                                                                                        content=[
                                                                                                            FileSearchResultContent(
                                                                                                                text='the network at most one minute after the cellular WAN connection goes up\n\nStandby State This option allows you to choose whether to remain connected or disconnected when this WAN connection is no\n\nlonger in the highest priority and has entered the standby state. When Remain connected is chosen, bringing up\n\nthis WAN connection to active makes it immediately available for use.\n\nIdle Disconnect If this is checked, the connection will disconnect when idle after the configured Time value.\n\nThis option is disabled by default.\n\nReply to ICMP\n\nPING\n\nIf the checkbox is unticked, this option is disabled and the system will not reply to any ICMP ping echo requests to\n\nthe WAN IP addresses of this WAN connection.\n\nDefault: ticked (Yes)\n\nCellular Settings\n\n\n\n114\n\nSIM Card If “Alternate between SIM A and SIM B periodically” is selected, the SIM card will be switching according to the\n\nschedule time in the SIM Cards Alternate.\n\nIf “Custom Selection” is selected, you can designate the priority of the SIM cards (SIM A/ SIM B/ Remote SIM/\n\nSpeedFusion Connect) and connect to.\xa0\n\nFor routers that support the SIM Injector, you may select the “Remote SIM” to provision a SIM from a SIM Injector.\n\nFurther details on the SIM Injector found is available here: https://www.peplink.com/products/sim-injector/\n\n(https://www.peplink.com/products/sim-injector/).\xa0\xa0\n\nRemote SIM\n\nSettings\n\nIf “Use Remote SIM Only” is selected in the SIM card section, the Remote SIM Settings will be shown.\xa0\n\nYou may need to enable the remote SIM Host settings in the Remote SIM management, see the section 22.10 or\n\nAppendix B for more details on FusionSIM. After that, click on “Scan nearby remote SIM server” to show the serial\n\nnumber(s) of the connected SIM Injector(s).\n\nIf you want to select a specific SIM, in the Cellular Settings, type “:” and then the number of the SIM slot, eg.1111-\n\n2222-3333:7.\xa0\n\nFallback to\n\nPrefered SIM\n\nwhen\n\nThis option is allowing to switch to another SIM cards when the Cellular WAN reached fallback timeout.\xa0\n\nSIM Cards\n\nAlternate\n\nIf “Alternate between SIM A and SIM B periodically” is selected in the SIM Card section, the SIM Cards Alternate\n\nwill be shown:\xa0\n\nYou may set the schedule time for for switching between SIM A only and SIM B only.\xa0\n\n5G/LTE/3G This drop-down menu allows restricting cellular to particular band. Click the  button to enable the selection of\n\nspecific bands.\n\nOptimal Network\n\nDiscovery\n\nCellular WANs by default will only handover from 3G to LTE network when there is no active data traffic, enable this\n\noption will make it run the handover procedures after fallback to 3G for a defined effective period, even this may\n\ninterrupt the connectivity for a short while.\n\nBand Selection When set to Auto, band selection allows for automatically connecting to available, supported bands (frequencies) .\n\nWhen set to Manual, you can manually select the bands (frequencies) the SIM will connect to.\n\nData Roaming This checkbox enables data roaming on this particular SIM card. When data roaming is enabled this option allows\n\nyou to select in which countries the SIM has a data connection. The option is configured by using MMC (country)\n\ncodes.Please check your service provider’s data roaming policy before proceeding.\n\nAuthentication Choose from PAP Only or CHAP Only to use those authentication methods exclusively. Select Auto to automatically\n\nchoose an authentication method.\n\nOperator Settings This setting allows you to configure the APN settings of your connection. If Auto is selected, the mobile operator\n\nshould be detected automatically. The connected device will be configured and connection will be made',
                                                                                                                type='text')]),
                                                                                                    FileSearchResult(
                                                                                                        file_id='file-XAov3nrJbk2byvhzCD282p',
                                                                                                        file_name='peplink_max_user_manual_peplink_documentation.pdf',
                                                                                                        score=0.6770760418061655,
                                                                                                        content=[
                                                                                                            FileSearchResultContent(
                                                                                                                text='135\n\nPhase 2 PFS Group Perfect forward secrecy (PFS) ensures that if a key was compromised, the attacker will be able to access only the\n\ndata protected by that key.\n\nNone – Do not request for PFS when initiating connection. However, since there is no valid reason to refuse PFS,\n\nthe system will allow the connection to use PFS if requested by the remote peer. This is the default value.\n\nGroup 2: 1024-bit Diffie-Hellman group. The larger the group number, the higher the security.\xa0\n\nGroup 5: 1536-bit is the third option.\n\nPhase 2 SA Lifetime This setting specifies the lifetime limit of this Phase 2 Security Association. By default, it is set at 28800 seconds.\n\nWAN Connection Priority\n\nWAN Connection Select the appropriate WAN connection from the drop-down menu.\n\nGRE Tunnel\n\nGeneric Routing Encapsulation (GRE) is a tunneling protocol that can encapsulate a wide variety of network layer protocols inside virtual point-\n\nto-point links over an Internet Protocol network. A GRE tunnel is similar to IPSec or SpeedFusion VPN.\n\nTo configure a GRE Tunnel, navigate to Advanced > GRE Tunnel.\n\nClick the New Profile button to create new GRE tunnel profiles that establish tunnel connections to remote tunnel endpoints via available WAN\n\nconnections. To edit the profiles, click on its associated connection name in the leftmost column.\n\nGRE Tunnel Profile Settings\n\n\n\n136\n\nName This field is for specifying a name to represent this GRE Tunnel connection profile.\xa0\n\nActive When this box is checked, this GRE Tunnel connection profile will be enabled. Otherwise, it will be disabled.\n\nRemote GRE IP Address This field is for entering the remote GRE’s IP address\xa0\n\nTunnel Local IP Address This field is for specifying the tunnel source IP address.\n\nTunnel Remote IP Address This field is for specifying the tunnel destination IP address\xa0\n\nTunnel Subnet Mask This field is to select the subnet mask that is to be used for the GRE tunnel.\xa0\xa0\n\nConnection Select the appropriate WAN connection from the drop-down menu.\n\nRemote Networks Input the LAN and subnets that are located at the remote site here.\n\nCh12. OpenVPN\n\nOpenVPN is a site to site VPN mode that can encapsulate a wide variety of network layer protocols inside virtual point-to-point links over an\n\nInternet Protocol network.\xa0\n\nTo configure a OpenVPN, navigate to Advanced > OpenVPN and click the New Profile.\xa0\xa0\n\nOpenVPN Profile Settings\n\nName This field is for specifying a name to represent this OpenVPN profile.\xa0\n\nActive When this box is checked, this OpenVPN connection profile will be enabled. Otherwise, it will be disabled.\n\nOpenVPN Profile Upload the OpenVPN configuration (.ovpn) file from your service provider.\n\nLogin Credential\n\n(Optional)\n\nThis option is an optional for you to enter the username and password to login for the OpenVPN connection if\n\nthe profile need to login.\xa0\n\nConnection Select the appropriate WAN connection from the drop-down menu.\n\n\n\n137\n\nCh13. Outbound Policy\n\nPepwave routers can flexibly manage and load balance outbound traffic among WAN connections.\xa0\n\nThe settings for managing and load balancing outbound traffic are located at\n\nAdvanced > Outbound Policy.\n\nAdding Rules for Outbound Policy\n\nThe menu underneath enables you to define Outbound policy rules:\n\nThe bottom-most rule is Default. Edit this rule to change the device’s default manner of controlling outbound traffic for all connections that do\n\nnot match any of the rules above it. Under the Service heading, click Default to change these settings.\xa0\n\nTo rearrange the priority of outbound rules, drag and drop them into the desired sequence.\n\n\n\n138\n\nBy default, Auto is selected as the Default Rule. You can select Custom to change the algorithm to be used. Please refer to the upcoming',
                                                                                                                type='text')]),
                                                                                                    FileSearchResult(
                                                                                                        file_id='file-XAov3nrJbk2byvhzCD282p',
                                                                                                        file_name='peplink_max_user_manual_peplink_documentation.pdf',
                                                                                                        score=0.673499499726334,
                                                                                                        content=[
                                                                                                            FileSearchResultContent(
                                                                                                                text='The default value is 48μs.\n\n\n\n203\n\nA – Advanced feature. Click the \xa0button on the top right-hand corner to activate.\n\nImportant Note\n\nPer FCC regulation, the country selection is not available on all models marketed in the US. All US models are fixed to US channels only.\n\nThe device with integrated AP can operate under the Wi-Fi Operating Mode, and the default setting is WAN + AP mode:\n\nNote: This option is available for selected devices only (HD2/HD4 and HD2/HD4 MBX).\xa0\n\nIntegrated AP\n\nWAN In this mode, all Wi-Fi will operate as Wi-Fi WAN and no integrated Wi-Fi AP will be operated on this device.\n\nIf Wi-Fi Operating mode is choosing WAN, The status indicated by the front panel LED is as follows:\xa0\n\nWi-Fi 1 is Green if Wi-Fi WAN 1 is enabled.\n\nWi-Fi 2 is Green if Wi-Fi WAN 2 is enabled.\xa0\n\nWAN +\n\nAP\n\nIn this mode, some Wi-Fi will operate as Wi-Fi WAN. Some other Wi-Fi WANs will be forced offline and their Wi-Fi resources will\n\nbe reserved for integrated Wi-Fi AP operations.\n\nIf Wi-Fi Operating mode is choosing WAN + AP, The status indicated by the front panel LED is as follows:\xa0\n\nWi-Fi 1 is Green if WI-FI WAN is enabled.\xa0\n\nWi-Fi 2 is Green if Wi-Fi AP is ON.\n\nAP In this mode, all Wi-Fi functions as integrated Wi-Fi AP. All Wi-Fi WANs will be forced to go offline.\n\nIf Wi-Fi Operating mode is choosing AP, The status indicated by the front panel LED is as follows:\xa0\n\nW-Fi 1 is Green, if there is any Wireless SSID is selected 2.4GHz.\xa0\n\nW-Fi 2 is Green, if there is any Wireless SSID is selected 5GHz.\xa0\n\nWeb Administration Settings (on External AP)\n\nEnable Check the box to allow the Pepwave router to manage the web admin access information of the AP.\n\nWeb Access Protocol These buttons specify the web access protocol used for accessing the web admin of the AP. The two available\n\noptions are HTTP and HTTPS.\n\nManagement Port This field specifies the management port used for accessing the device.\n\nHTTP to HTTPS\n\nRedirection\n\nThis option will be available if you have chosen HTTPS as the Web Access Protocol. With this enabled, any HTTP\n\naccess to the web admin will redirect to HTTPS automatically.\n\n\n\n204\n\nAdmin User Name This field specifies the administrator username of the web admin. It is set as admin by default.\n\nAdmin Password This field allows you to specify a new administrator password. You may also click the Generate button and let the\n\nsystem generate a random password automatically.\n\nThis allows users to configure AP Time Settings (both Timezone and NTP) in AP Controller.\n\nAP Time Settings\n\nTime Zone This field is to select the time zone for the AP controller.\xa0\n\nTime Server This field is to select the time server for the AP controller.\n\nThis settings is to allow user to manage external AP’s controller unreachable action. When Manage Unreachable Action is checked, there will\n\nhave 2 options which are “None” and “Radio Off“.\xa0\n\nThis is an option to enable client load balancing for AP Controller. When the option is enabled, it is trying to balance the station count on APs\n\nwithin the same profile.\xa0\n\nSome Pepwave models displays a screen similar to the one shown below, navigating to AP > Settings:\xa0\n\nWi-Fi Radio Settings\n\nOperating Country This option sets the country whose regulations the Pepwave router follows.\xa0\n\nWi-Fi Antenna Wi-Fi Antenna Choose from the router’s internal or optional external antennas, if so equipped.\xa0\n\nWi-Fi AP Settings\n\n\n\n205',
                                                                                                                type='text')]),
                                                                                                    FileSearchResult(
                                                                                                        file_id='file-XAov3nrJbk2byvhzCD282p',
                                                                                                        file_name='peplink_max_user_manual_peplink_documentation.pdf',
                                                                                                        score=0.6513486598411107,
                                                                                                        content=[
                                                                                                            FileSearchResultContent(
                                                                                                                text='which can enhance security.\xa0Traffic is passed to the upper communication layer(s).\xa0By default, the setting is disabled.\xa0\n\nMaximum\n\nNumber of\n\nClients\n\nIndicate the maximum number of clients that should be able to connect to each frequency.\n\n^ Advanced feature. Click the  button on the top right-hand corner to activate.\n\nSecurity Settings\n\nSecurity\n\nPolicy\n\nThis setting configures the wireless authentication and encryption methods. Available options are:\n\nOpen (No Encryption)\n\nEnhanced Open (OWE)\n\nWPA3 – Personal (AES:CCMP)\n\nWPA3 – Enterprise (AES:CCMP)\xa0\n\nWPA2/WPA3 – Personal (AES:CCMP)\n\nWPA2 – Personal (AES:CCMP)\xa0\n\nWPA2 – Enterprise (AES:CCMP)\xa0\n\nWPA/WPA2 – Personal (TKIP/AES: CCMP)\n\nWPA/WPA2 – Enterprise (TKIP/AES: CCMP)\n\nTo allow any Wi-Fi client to access your AP without authentication, select Open (No Encryption). Details of each available\n\nauthentication method follow.\n\nEnhanced Open (OWE)\n\nTransition\n\nMode\n\nWith the option enabled, legacy clients will be able to connect in Open mode. With the option disabled, legacy clients will\n\nnot be able to connect. OWE-supported clients will always connect in OWE mode.\n\n\n\n195\n\nWPA3 – Personal\n\nShared Key Enter a passphrase of between 8 and 63 alphanumeric characters to create a passphrase used for data encryption and\n\nauthentication. Click Hide / Show Characters to toggle visibility.\n\nFast\n\nTransition\n\nFast Transition\n\n[802.11r] When this option is ticked, the transition process for a mobile client is improved as it moves between access\n\npoints.\n\nWPA3 – Enterprise\n\n802.1X\n\nVersion\n\nChoose v1 or v2 of the 802.1x EAPOL. When v1 is selected, both v1 and v2 clients can associate with the access point. When\n\nv2 is selected, only v2 clients can associate with the access point. Most modern wireless clients support v2. For stations that\n\ndo not support v2, select v1. The default is v2.\n\nFast\n\nTransition\n\nFast Transition\n\n[802.11r] When this option is ticked, the transition process of a mobile client is improved as it moves between access points.\n\nWPA2/WPA3 – Personal\n\nShared Key Enter a passphrase of between 8 and 63 alphanumeric characters to create a passphrase used for data\n\nencryption and authentication. Click Hide / Show Characters to toggle visibility.\n\nManagement Frame\n\nProtection\n\nThis feature protects stations against forged management frames spoofed from other devices. Frames that are\n\nprotected include Disassociation, Deauthentication, and QoS Action.\n\nFast Transition Fast Transition\n\n[802.11r] When this option is ticked, the transition process of a mobile client is improved as it moves between\n\naccess points.\n\n\n\n196\n\nWPA2 – Personal\n\nPrivate Shared Key Enable the Private Pre-Shared Key (PPSK) option to set unique pre-shared keys for individual users or groups on\n\nthe same SSID. Please provide the following information for each user or group:\n\nName\n\n(Optional)\n\nName for the profile\n\nShared Key The passphrase key is used for data encryption and authentication. You can click\n\n“Hide/Show Characters” to toggle the visibility of the shared key.\n\nMAC\n\n(Optional)\n\nBy default, all MAC addresses are allowed. If a specific MAC address is entered, only that\n\ndevice can access this WiFi SSID using the provided shared key.\n\nVLAN ID\n\n(Optional)\n\nThe VLAN is assigned to the user upon logging into the WiFi. If not specified, the user will',
                                                                                                                type='text')]),
                                                                                                    FileSearchResult(
                                                                                                        file_id='file-XAov3nrJbk2byvhzCD282p',
                                                                                                        file_name='peplink_max_user_manual_peplink_documentation.pdf',
                                                                                                        score=0.602763343340358,
                                                                                                        content=[
                                                                                                            FileSearchResultContent(
                                                                                                                text='Access Point (https://manual.peplink.com/peplink-max-user-manual/#1919)\n\nWireless SSID (https://manual.peplink.com/peplink-max-user-manual/#1930)\n\nWireless Client (https://manual.peplink.com/peplink-max-user-manual/#1934)\n\nMesh / WDS (https://manual.peplink.com/peplink-max-user-manual/#1932)\n\nNearby Device (https://manual.peplink.com/peplink-max-user-manual/#1939)\n\nEvent Log (https://manual.peplink.com/peplink-max-user-manual/#1942)\n\nInfo\n\nA comprehensive overview of your AP can be accessed by navigating to AP > Controller Status > Info.\n\n\n\n206\n\nAP Controller\n\nLicense\n\nLimit\n\nThis field displays the maximum number of AP your Balance router can control. You can purchase licenses to increase the\n\nnumber of AP you can manage.\n\nFrequency Underneath, there are two check boxes labeled 2.4 Ghz and 5 Ghz. Clicking either box will toggle the display of information\n\nfor that frequency. By default, the graphs display the number of clients and data usage for both 2.4GHz and 5 GHz\n\nfrequencies.\xa0\n\nSSID The colored boxes indicate the SSID to display information for. Clicking any colored box will toggle the display of information\n\nfor that SSID. By default, all the graphs show information for all SSIDs.\n\nNo. of APs This pie chart and table indicates how many APs are online and how many are offline.\xa0\n\nNo.of\n\nClients\n\nThis graph displays the number of clients connected to each network at any given time. Mouse over any line on the graph to\n\nsee how many clients connected to a specific SSID for that point in time.\n\nData\n\nUsage\n\nThis graph enables you to see the data usage of any SSID for any given time period. Mouse over any line on the graph to see\n\nthe data usage by each SSID for that point in time. Use the buttons next to Zoom to select the time scale you wish to view.\n\nIn addition, you could use the sliders at the bottom to further refine your timescale.\n\n\n\n207\n\n0\n\nEvents\n\nThis event log displays all activity on your AP network, down to the client level. Click View Alerts to see only alerts, and click the More… link\n\nfor additional records.\n\nThis allow user to configure AP Time Settings (both Timezone and NTP) in AP Controller.\n\nAP Time Settings\n\nTime Zone Ths field is to select the time zone for the AP controller.\xa0\n\nTime Server Ths field is to select the time server for the AP controller.\n\nThis settings is to allow user to manage external AP’s controller unreachable action. When Manage Unreachable Action is checked, there will\n\nhave 2 options which are “None” and “Radio Off“.\xa0\n\nThis is an option to enable client load balancing for AP Controller. When the option is enabled, it is trying to balance the station count on APs\n\nwithin the same profile.\xa0\n\nAccess Point\n\nA detailed breakdown of data usage for each AP is available at AP > Controller Status > Access Point.\n\n\n\n208\n\nManaged\xa0 APs\n\n\n\n209\n\nManaged\n\nAPs\n\nThis table shows the detailed information on each AP, including channel, number of clients, upload traffic, and download\n\ntraffic. Click the blue arrows at the left of the table to expand and collapse information on each device group.\n\nOn the right of the table, you will see the following icons: .\n\nClick the \xa0icon to see a usage table for each client:\n\nClick the  icon to configure each client',
                                                                                                                type='text')]),
                                                                                                    FileSearchResult(
                                                                                                        file_id='file-XAov3nrJbk2byvhzCD282p',
                                                                                                        file_name='peplink_max_user_manual_peplink_documentation.pdf',
                                                                                                        score=0.5844965756731234,
                                                                                                        content=[
                                                                                                            FileSearchResultContent(
                                                                                                                text='Add Custom mobile operator settings (APN, user name, password).\n\n4. Repeat configuration for all SIM cards which need custom settings.\n\n\n\n252\n\n5. Click Apply Changes to take effect.\n\nScenario 2\n\nSIM Injector in WAN of main Router and multiple Cellular Routers\n\nSetup topology\n\nIn this scenario, each HD Dome creates a WAN connection to the main router. A single SIM Injector is used to provide SIM cards for each HD\n\nDome. The HD Dome can be replaced with any Peplink cellular router supporting RemoteSIM technology.\n\nThis scenario requires the completion of the configuration steps shown in Scenario 1 in addition to the configuration steps explained\n\nbelow.\n\nAdditional configurations for Cellular Routers\n\nStep 1.\xa0Disable the DHCP server.\n\nHD Dome 1 should act as a DHCP server.\n\nHD Dome 2 should be configured to have a static IP address with DHCP disabled.\n\nBoth routers should be in the same subnet (e.g. 192.168.50.1 and 192.168.50.2).\n\n1. Go to\xa0Network\xa0(Top tab), then\xa0Network Settings\xa0(Left-side tab), and click on\xa0Untagged LAN. This will open up the LAN settings page.\n\n2. Change the IP address to 192.168.50.2.\n\n3. In the\xa0DHCP Server\xa0section, uncheck the checkbox to disable DHCP Server.\n\n4. Click\xa0Save\xa0and\xa0Apply Changes.\n\nStep 2.\xa0Ethernet port configuration\n\n\n\n253\n\nThe Ethernet port must be set to\xa0ACCESS\xa0mode for each HD Dome. To do this, dummy VLANs need to be created first.\n\n1. Go to\xa0Network\xa0(Top tab), then\xa0Network Settings\xa0(Left-side tab), and click on\xa0New LAN. This will open the settings page to create a dummy\n\nVLAN.\n\n2. The image below shows the values that need to be changed to create a new VLAN:\n\nNote: set different IP addresses for each HD dome (e.g. 192.168.10.1 and 192.168.10.2).\n\n3. Click Save and\xa0Apply Changes.\n\n4. Go to\xa0Network\xa0(Top tab), then\xa0Port Settings\xa0(Left-side tab).\n\n5. Set the Port Type to\xa0Access\xa0and set VLAN to\xa0Untagged LAN (see picture below).\n\n6. Click\xa0Save\xa0and\xa0Apply Changes.\n\nConfiguration requirements for the main Router\n\nRequirements for the main router are:\n\nConfigure\xa0WAN 1\xa0as a DHCP client.\n\nWAN 1\xa0will automatically get the Gateway IP address from HD Dome 1.\n\nConfigure\xa0WAN 2\xa0as a Static IP and set it to 192.168.50.12.\n\nConfigure\xa0WAN 2\xa0Gateway to 192.168.50.2. Same as the HD Dome 2’s IP address.\n\nScenario 3\n\nSIM Injector in LAN of main Router and multiple Cellular Routers\n\n\n\n254\n\nSetup topology\n\nIn this scenario, SIMs are provided to the HD Domes via the main router. In this example, the Remote SIM Proxy functionality needs to be\n\nenabled on the main router.\n\nNotes:\xa0\n\nHD Dome can be replaced with any other cellular router that supports RemoteSIM.\n\nIt is recommended to use Peplink Balance series (https://www.peplink.com/products/balance-series/) or X series\n\n(https://www.peplink.com/products/x-series/) routers as the main router.\n\nThis scenario requires the completion of the configuration steps for the cellular router and the SIM Injector as in Scenario 1. The\n\nconfiguration for the main router is explained below.\n\nMain Router configuration',
                                                                                                                type='text')]),
                                                                                                    FileSearchResult(
                                                                                                        file_id='file-XAov3nrJbk2byvhzCD282p',
                                                                                                        file_name='peplink_max_user_manual_peplink_documentation.pdf',
                                                                                                        score=0.5815148613145121,
                                                                                                        content=[
                                                                                                            FileSearchResultContent(
                                                                                                                text='Filter Mode This field allows for the selection of the filter mode for route import.\n\nNone: All BGP routes will be accepted.\n\nAccept: Routes in “Restricted Networks” will be accepted, routes not in the list will be rejected.\n\nReject: Routes in “Blocked Networks” will be rejected, routes not in the list will be accepted.\n\nRestricted Networks / Blocked\n\nNetworks\n\nThis field specifies the network(s) in the “route import” entry.\n\nExact Match: When this box is checked, only routes with the same Network and Subnet Mask\n\nwill be filtered.\n\nOtherwise, routes within the Networks and Subnets will be filtered.\n\n\n\n172\n\nFilter Mode This field allows for the selection of the filter mode for route export.\n\nNone: All BGP routes will be accepted.\n\nAccept: Routes in “Restricted Networks” will be accepted, routes not in the list will be rejected.\n\nReject: Routes in “Blocked Networks” will be rejected, routes not in the list will be accepted.\n\nRestricted Networks / Blocked\n\nNetworks\n\nThis field specifies the network(s) in the “route export” entry.\n\nExact Match: When this box is checked, only routes with the same Network and Subnet Mask\n\nwill be filtered.\n\nOtherwise, routes within the Networks and Subnets will be filtered.\n\nExport to other BGP Profile When this box is checked, routes learnt from this BGP profile will be exported to other BGP\n\nprofiles.\n\nExport to OSPF When this box is checked, routes learnt from this BGP profile will be exported to the OSPF\n\nrouting protocol.\n\nCh23. Remote User Access\n\nA remote-access VPN connection allows an individual user to connect to a private business network from a remote location using a laptop or\n\ndesktop computer connected to the Internet. Networks routed by a Pepwave router can be remotely accessed via OpenVPN, L2TP with IPsec or\n\nPPTP. To configure this feature, navigate to Advanced > Remote User Access and choose the required VPN type.\n\nRemote User Access Settings\n\nEnable When this box is checked, this Remote User Access profile will be enabled.\n\nIf it is left unchecked, it will be disabled.\n\n\n\n173\n\nVPN Type This field allows you to select the VPN type for the remote user access connection. The available options are:\xa0\n\nL2TP with IPsec\n\nIf L2TP with IPsec is selected, it may need to enter the pre-shared key for the remote user access.\xa0\n\nPPTP\xa0\n\nIf PPTP selected, there is no additional configuration required. The Point-to-Point Tunneling Protocol (PPTP) is an\n\nobsolete method for implementing virtual private networks. PPTP has many well known security issues\n\nOpenVPN\n\nIf the OpenVPN is selected, the OpenVPN Client\xa0 profile can be downloaded from the Status > Device page after\n\nthe configuration has been saved.\n\nYou have a choice between 2 different OpenVPN Client profiles:\n\n“Route all traffic” profile\n\nUsing this profile, VPN clients will send all the traffic through the OpenVPN tunnel\n\n“Split tunnel” profile\n\nUsing this profile, VPN clients will ONLY send those traffic designated to the untagged LAN and VLAN segment\n\nthrough the OpenVPN tunnel.\n\nPre-shared Key If L2TP with IPsec is selected in the VPN Type, enter the pre shared key in the text field. Please note that remote\n\ndevices will need this preshared key to access the Balance.\n\nDisabled Weak\n\nCiphers\n\nYou may click the \xa0button to show in the Pre-shared key and enable this option.\n\nWhen checked, weak ciphers such as 3DES will be disabled.\n\nPlease note: Legacy and Android devices may not able to connect.\n\nConnection Security\n\nRefresh',
                                                                                                                type='text')])]),
                                                                         type='file_search')], type='tool_calls'),
        thread_id='thread_tEszx5NNXbL5nHIdj5DIw8Dw',
        type='tool_calls',
        usage=Usage(completion_tokens=0, prompt_tokens=0, total_tokens=0, prompt_token_details={'cached_tokens': 0}),
        expires_at=None)
